## การฝังข้อมูล

ในตัวอย่างก่อนหน้านี้ เราได้ทำงานกับเวกเตอร์แบบถุงคำ (bag-of-words) ที่มีมิติสูงและมีความยาวเท่ากับ `vocab_size` โดยเราได้แปลงจากเวกเตอร์ตัวแทนตำแหน่งที่มีมิติต่ำไปเป็นตัวแทนแบบ one-hot ที่มีความเบาบาง (sparse) อย่างชัดเจน อย่างไรก็ตาม ตัวแทนแบบ one-hot นี้ไม่ประหยัดหน่วยความจำ และนอกจากนี้ แต่ละคำยังถูกพิจารณาแยกจากกันโดยสิ้นเชิง กล่าวคือ เวกเตอร์ที่เข้ารหัสแบบ one-hot ไม่ได้แสดงถึงความคล้ายคลึงทางความหมายระหว่างคำต่าง ๆ

ในหน่วยนี้ เราจะสำรวจชุดข้อมูล **News AG** ต่อไป เพื่อเริ่มต้น เรามาโหลดข้อมูลและดึงคำจำกัดความบางส่วนจากสมุดบันทึกก่อนหน้านี้กัน


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## การฝังข้อมูลคืออะไร?

แนวคิดของ **การฝังข้อมูล** คือการแทนคำด้วยเวกเตอร์ที่มีมิติที่ต่ำกว่าและหนาแน่น ซึ่งสะท้อนถึงความหมายเชิงความหมายของคำในบางรูปแบบ เราจะพูดถึงวิธีการสร้างการฝังคำที่มีความหมายในภายหลัง แต่ตอนนี้ให้คิดว่าการฝังข้อมูลเป็นวิธีลดมิติของเวกเตอร์คำ

ดังนั้น ชั้นการฝังข้อมูลจะรับคำเป็นข้อมูลเข้า และสร้างเวกเตอร์ผลลัพธ์ที่มีขนาด `embedding_size` ที่กำหนดไว้ ในแง่หนึ่ง มันคล้ายกับชั้น `Linear` แต่แทนที่จะรับเวกเตอร์ที่เข้ารหัสแบบ one-hot มันจะสามารถรับหมายเลขคำเป็นข้อมูลเข้าได้

โดยการใช้ชั้นการฝังข้อมูลเป็นชั้นแรกในเครือข่ายของเรา เราสามารถเปลี่ยนจากโมเดล bag-of-words ไปเป็นโมเดล **embedding bag** ซึ่งเราจะเปลี่ยนคำแต่ละคำในข้อความของเราให้เป็นการฝังข้อมูลที่สอดคล้องกัน และจากนั้นคำนวณฟังก์ชันรวมบางอย่างจากการฝังข้อมูลเหล่านั้น เช่น `sum`, `average` หรือ `max`

![ภาพแสดงตัวอย่างตัวจำแนกที่ใช้การฝังข้อมูลสำหรับคำในลำดับห้าคำ](../../../../../translated_images/th/embedding-classifier-example.b77f021a7ee67eee.webp)

เครือข่ายประสาทเทียมของเราจะเริ่มต้นด้วยชั้นการฝังข้อมูล ตามด้วยชั้นการรวม และตัวจำแนกเชิงเส้นที่อยู่ด้านบน


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### การจัดการกับขนาดลำดับตัวแปร

จากสถาปัตยกรรมนี้ ทำให้เราจำเป็นต้องสร้าง minibatches สำหรับเครือข่ายของเราในรูปแบบที่เฉพาะเจาะจง ในหน่วยก่อนหน้า เมื่อเราใช้ bag-of-words ขนาดของ BoW tensors ในแต่ละ minibatch จะมีขนาดเท่ากันคือ `vocab_size` โดยไม่คำนึงถึงความยาวจริงของลำดับข้อความของเรา แต่เมื่อเราเปลี่ยนมาใช้ word embeddings เราจะพบว่าจำนวนคำในแต่ละตัวอย่างข้อความอาจแตกต่างกัน และเมื่อรวมตัวอย่างเหล่านั้นเข้าด้วยกันใน minibatches เราจำเป็นต้องใช้การเติมค่า (padding)

สิ่งนี้สามารถทำได้โดยใช้เทคนิคเดียวกันกับการให้ฟังก์ชัน `collate_fn` กับ datasource:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### การฝึกตัวแยกประเภทแบบฝังตัว

ตอนนี้เมื่อเราได้กำหนด dataloader ที่เหมาะสมแล้ว เราสามารถฝึกโมเดลได้โดยใช้ฟังก์ชันการฝึกที่เราได้กำหนดไว้ในหน่วยก่อนหน้า:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **หมายเหตุ**: ที่นี่เรากำลังฝึกเพียง 25,000 รายการ (น้อยกว่าหนึ่ง epoch เต็ม) เพื่อประหยัดเวลา แต่คุณสามารถฝึกต่อไปได้ เขียนฟังก์ชันเพื่อฝึกหลายๆ epoch และทดลองปรับพารามิเตอร์อัตราการเรียนรู้เพื่อให้ได้ความแม่นยำที่สูงขึ้น คุณควรจะสามารถเพิ่มความแม่นยำได้ถึงประมาณ 90%


### ชั้น EmbeddingBag และการแสดงผลลำดับที่มีความยาวแปรผัน

ในสถาปัตยกรรมก่อนหน้านี้ เราจำเป็นต้องเติมข้อมูลในทุกลำดับให้มีความยาวเท่ากันเพื่อให้สามารถใส่ลงในชุดข้อมูลย่อยได้ วิธีนี้ไม่ใช่วิธีที่มีประสิทธิภาพที่สุดในการแสดงผลลำดับที่มีความยาวแปรผัน - อีกวิธีหนึ่งคือการใช้ **เวกเตอร์ออฟเซ็ต** ซึ่งจะเก็บค่าตำแหน่งเริ่มต้นของลำดับทั้งหมดที่ถูกจัดเก็บในเวกเตอร์ขนาดใหญ่หนึ่งตัว

![ภาพแสดงการแสดงผลลำดับแบบออฟเซ็ต](../../../../../translated_images/th/offset-sequence-representation.eb73fcefb29b46ee.webp)

> **Note**: ในภาพด้านบน เราแสดงลำดับของตัวอักษร แต่ในตัวอย่างของเรา เรากำลังทำงานกับลำดับของคำ อย่างไรก็ตาม หลักการทั่วไปในการแสดงผลลำดับด้วยเวกเตอร์ออฟเซ็ตยังคงเหมือนเดิม

ในการทำงานกับการแสดงผลแบบออฟเซ็ต เราใช้ชั้น [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html) ซึ่งคล้ายกับ `Embedding` แต่จะรับเวกเตอร์เนื้อหาและเวกเตอร์ออฟเซ็ตเป็นข้อมูลนำเข้า และยังมีชั้นการเฉลี่ยที่สามารถเป็น `mean`, `sum` หรือ `max` ได้

นี่คือตัวอย่างเครือข่ายที่ปรับปรุงแล้วซึ่งใช้ `EmbeddingBag`:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

ในการเตรียมชุดข้อมูลสำหรับการฝึก เราจำเป็นต้องจัดเตรียมฟังก์ชันการแปลงที่จะเตรียมเวกเตอร์ออฟเซ็ต:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

โปรดทราบว่าแตกต่างจากตัวอย่างก่อนหน้านี้ทั้งหมด เครือข่ายของเราตอนนี้รับพารามิเตอร์สองตัว: เวกเตอร์ข้อมูลและเวกเตอร์ออฟเซ็ต ซึ่งมีขนาดต่างกัน เช่นเดียวกัน ตัวโหลดข้อมูลของเรายังให้ค่ามา 3 ค่าแทนที่จะเป็น 2: ทั้งเวกเตอร์ข้อความและเวกเตอร์ออฟเซ็ตถูกจัดเตรียมเป็นฟีเจอร์ ดังนั้นเราจำเป็นต้องปรับฟังก์ชันการฝึกของเราเล็กน้อยเพื่อจัดการกับสิ่งนี้:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## การฝังความหมาย: Word2Vec

ในตัวอย่างก่อนหน้านี้ เลเยอร์ฝังตัวแบบจำลองได้เรียนรู้ที่จะจับคู่คำกับการแสดงผลในรูปแบบเวกเตอร์ อย่างไรก็ตาม การแสดงผลนี้ไม่ได้มีความหมายในเชิงความหมายมากนัก จะดีกว่าหากเราสามารถเรียนรู้การแสดงผลในรูปแบบเวกเตอร์ที่คำที่มีความหมายคล้ายกันหรือคำพ้องความหมายจะถูกจับคู่กับเวกเตอร์ที่อยู่ใกล้กันในแง่ของระยะเวกเตอร์บางประเภท (เช่น ระยะทางแบบยุคลิด)

เพื่อทำเช่นนั้น เราจำเป็นต้องฝึกฝนแบบจำลองการฝังตัวของเราล่วงหน้าด้วยชุดข้อความขนาดใหญ่ในวิธีการเฉพาะ หนึ่งในวิธีแรก ๆ ในการฝึกฝนการฝังความหมายเรียกว่า [Word2Vec](https://en.wikipedia.org/wiki/Word2vec) ซึ่งอิงตามสถาปัตยกรรมหลักสองแบบที่ใช้ในการสร้างการแสดงผลแบบกระจายของคำ:

 - **Continuous bag-of-words** (CBoW) — ในสถาปัตยกรรมนี้ เราฝึกแบบจำลองให้ทำนายคำจากบริบทโดยรอบ โดยให้ ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$ เป้าหมายของแบบจำลองคือการทำนาย $W_0$ จาก $(W_{-2},W_{-1},W_1,W_2)$
 - **Continuous skip-gram** ตรงข้ามกับ CBoW แบบจำลองจะใช้หน้าต่างบริบทของคำรอบ ๆ เพื่อทำนายคำปัจจุบัน

CBoW ทำงานได้เร็วกว่า ในขณะที่ skip-gram ช้ากว่า แต่สามารถแสดงคำที่พบได้น้อยได้ดีกว่า

![ภาพแสดงอัลกอริทึม CBoW และ Skip-Gram สำหรับการแปลงคำเป็นเวกเตอร์](../../../../../translated_images/th/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

เพื่อทดลองใช้การฝัง Word2Vec ที่ได้รับการฝึกฝนล่วงหน้าบนชุดข้อมูล Google News เราสามารถใช้ไลบรารี **gensim** ด้านล่างนี้คือตัวอย่างการค้นหาคำที่คล้ายกับ 'neural' มากที่สุด

> **Note:** เมื่อคุณสร้างเวกเตอร์คำครั้งแรก การดาวน์โหลดอาจใช้เวลาสักครู่!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


เรายังสามารถคำนวณเวกเตอร์เอมเบดดิ้งจากคำ เพื่อใช้ในการฝึกโมเดลการจำแนกประเภท (เราแสดงเฉพาะ 20 องค์ประกอบแรกของเวกเตอร์เพื่อความชัดเจน):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

ทั้ง CBoW และ Skip-Grams เป็นการฝังคำแบบ “predictive” ซึ่งหมายความว่ามันพิจารณาเฉพาะบริบทในพื้นที่ใกล้เคียงเท่านั้น Word2Vec ไม่ได้ใช้ประโยชน์จากบริบทในภาพรวม

**FastText** พัฒนาต่อยอดจาก Word2Vec โดยการเรียนรู้การแสดงผลแบบเวกเตอร์สำหรับแต่ละคำและ n-grams ของตัวอักษรที่พบในแต่ละคำ ค่าของการแสดงผลเหล่านี้จะถูกเฉลี่ยเป็นเวกเตอร์เดียวในแต่ละขั้นตอนการฝึก แม้ว่าวิธีนี้จะเพิ่มการคำนวณเพิ่มเติมในขั้นตอนการฝึกเบื้องต้น แต่มันช่วยให้การฝังคำสามารถเข้ารหัสข้อมูลในระดับย่อยของคำได้

อีกวิธีหนึ่งคือ **GloVe** ซึ่งใช้แนวคิดของเมทริกซ์การเกิดร่วมกัน (co-occurrence matrix) และใช้วิธีการแบบประสาทเทียม (neural methods) ในการแยกเมทริกซ์การเกิดร่วมกันออกเป็นเวกเตอร์คำที่มีความหมายลึกซึ้งและไม่เป็นเชิงเส้นมากขึ้น

คุณสามารถทดลองเปลี่ยนการฝังคำเป็น FastText และ GloVe ได้ เนื่องจาก gensim รองรับโมเดลการฝังคำที่หลากหลาย


## การใช้ Pre-Trained Embeddings ใน PyTorch

เราสามารถปรับตัวอย่างด้านบนเพื่อเติมข้อมูลในเมทริกซ์ของ embedding layer ด้วย embedding ที่มีความหมาย เช่น Word2Vec เราต้องคำนึงถึงว่าคำศัพท์ของ pre-trained embedding และ text corpus ของเราอาจไม่ตรงกัน ดังนั้นเราจะกำหนดค่า weights สำหรับคำที่ขาดหายไปด้วยค่าที่สุ่มขึ้นมา:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

ในกรณีของเรา เราไม่เห็นการเพิ่มขึ้นของความแม่นยำอย่างมาก ซึ่งน่าจะเกิดจากคำศัพท์ที่แตกต่างกันมาก  
เพื่อแก้ปัญหาคำศัพท์ที่แตกต่างกัน เราสามารถใช้วิธีแก้ไขปัญหาดังต่อไปนี้:  
* ฝึกโมเดล word2vec ใหม่โดยใช้คำศัพท์ของเรา  
* โหลดชุดข้อมูลของเราด้วยคำศัพท์จากโมเดล word2vec ที่ถูกฝึกไว้ล่วงหน้า โดยสามารถกำหนดคำศัพท์ที่ใช้โหลดชุดข้อมูลได้ในระหว่างการโหลด  

วิธีหลังดูเหมือนจะง่ายกว่า โดยเฉพาะอย่างยิ่งเพราะ PyTorch `torchtext` framework มีการรองรับ embeddings ในตัวอยู่แล้ว เราสามารถสร้างคำศัพท์ที่ใช้ GloVe ได้ในลักษณะดังนี้:  


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


คำศัพท์ที่โหลดมามีการดำเนินการพื้นฐานดังนี้:
* พจนานุกรม `vocab.stoi` ช่วยให้เราสามารถแปลงคำให้เป็นดัชนีในพจนานุกรม
* `vocab.itos` ทำงานตรงกันข้าม - แปลงตัวเลขกลับเป็นคำ
* `vocab.vectors` คืออาร์เรย์ของเวกเตอร์ฝังตัว ดังนั้นเพื่อดึงเวกเตอร์ฝังตัวของคำ `s` เราจำเป็นต้องใช้ `vocab.vectors[vocab.stoi[s]]`

นี่คือตัวอย่างการจัดการเวกเตอร์ฝังตัวเพื่อแสดงสมการ **kind-man+woman = queen** (ฉันต้องปรับค่าสัมประสิทธิ์เล็กน้อยเพื่อให้มันทำงานได้):


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

ในการฝึกตัวจำแนกโดยใช้การฝังเหล่านั้น เราต้องเข้ารหัสชุดข้อมูลของเราก่อนโดยใช้คำศัพท์ของ GloVe:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

ดังที่เราได้เห็นข้างต้น เวกเตอร์เอมเบดดิ้งทั้งหมดถูกเก็บไว้ในเมทริกซ์ `vocab.vectors` ซึ่งทำให้ง่ายมากที่จะโหลดเวกเตอร์เหล่านั้นไปยังน้ำหนักของเลเยอร์เอมเบดดิ้งโดยการคัดลอกอย่างง่าย:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

หนึ่งในเหตุผลที่เราไม่เห็นการเพิ่มขึ้นของความแม่นยำอย่างมีนัยสำคัญเป็นเพราะคำบางคำจากชุดข้อมูลของเราหายไปจากคำศัพท์ที่ผ่านการฝึกอบรมล่วงหน้าของ GloVe และดังนั้นคำเหล่านั้นจึงถูกละเลยไป เพื่อเอาชนะปัญหานี้ เราสามารถฝึกฝนการฝังตัวคำของเราเองบนชุดข้อมูลของเรา


## การฝังบริบท (Contextual Embeddings)

ข้อจำกัดสำคัญอย่างหนึ่งของการฝังคำแบบดั้งเดิมที่ผ่านการฝึกอบรมล่วงหน้า เช่น Word2Vec คือปัญหาเรื่องการแยกความหมายของคำ (word sense disambiguation) แม้ว่าการฝังคำที่ผ่านการฝึกอบรมล่วงหน้าจะสามารถจับความหมายบางส่วนของคำในบริบทได้ แต่ความหมายทุกแบบของคำจะถูกเข้ารหัสไว้ในรูปแบบการฝังเดียวกัน สิ่งนี้อาจทำให้เกิดปัญหาในโมเดลที่ใช้งานต่อเนื่อง เนื่องจากคำหลายคำ เช่นคำว่า 'play' มีความหมายที่แตกต่างกันขึ้นอยู่กับบริบทที่ใช้งาน

ตัวอย่างเช่น คำว่า 'play' ในสองประโยคต่อไปนี้มีความหมายที่แตกต่างกันอย่างชัดเจน:
- ฉันไปดู **ละคร** ที่โรงละคร
- จอห์นอยากจะ **เล่น** กับเพื่อนของเขา

การฝังคำที่ผ่านการฝึกอบรมล่วงหน้าข้างต้นแสดงถึงความหมายทั้งสองของคำว่า 'play' ในรูปแบบการฝังเดียวกัน เพื่อแก้ไขข้อจำกัดนี้ เราจำเป็นต้องสร้างการฝังคำที่อิงตาม **โมเดลภาษา** ซึ่งได้รับการฝึกอบรมจากคลังข้อความขนาดใหญ่ และ *เข้าใจ* ว่าคำสามารถนำมาประกอบกันในบริบทที่แตกต่างกันได้อย่างไร การพูดถึงการฝังบริบทอยู่นอกขอบเขตของบทเรียนนี้ แต่เราจะกลับมาพูดถึงเรื่องนี้อีกครั้งเมื่อพูดถึงโมเดลภาษาในหน่วยถัดไป



---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้การแปลมีความถูกต้อง แต่โปรดทราบว่าการแปลอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่แม่นยำ เอกสารต้นฉบับในภาษาต้นทางควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ แนะนำให้ใช้บริการแปลภาษามนุษย์ที่เป็นมืออาชีพ เราไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความที่ผิดพลาดซึ่งเกิดจากการใช้การแปลนี้
